In [2]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import random
!pip install tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/ed/1a/b4ab4b8f8b3a41fade4899fd00b5b2d2dad0981f3e1bb10df4c522975fd7/tensorflow-2.15.0.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 211.6 kB/s eta 0:00:001m147.5 kB/s eta 0:00:01
  Obtaining dependency information for libclang>=13.0.0 from https://files.pythonhosted.org/packages/ea/df/55525e489c43f9dbb6c8ea27d8a567b3dcd18a22f3c45483055f5ca6611d/libclang-16.0.6-py2.py3-none-manylinux2010_x8

  Obtaining dependency information for cachetools<6.0,>=2.0.0 from https://files.pythonhosted.org/packages/a2/91/2d843adb9fbd911e0da45fbf6f18ca89d07a087c3daa23e955584f90ebf4/cachetools-5.3.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 708.6 kB/s eta 0:00:00 kB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:04m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 2.2 MB/s eta 0:00:002.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 2.9 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.9 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 2.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.5 MB/s eta 0:00:000:00:010:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:01
  

2023-12-15 14:25:52.160445: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-15 14:25:52.239767: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 14:25:52.239816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 14:25:52.242849: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-15 14:25:52.255330: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-15 14:25:52.256358: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [9]:
# Function to build DQN model
def build_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(24, input_dim=input_dim, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(output_dim, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())
    return model

# Function to train DQN policy
def train_dqn(env, episodes=1000, epsilon_decay=0.995):
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    batch_size = 8
    memory = deque(maxlen=2000)
    model = build_model(state_size, action_size)
    epsilon = 0.1

    rewards = []
    max_episode_length = 100
    for episode in range(episodes):
        state = env.reset()[0]
        state = np.reshape(state, [1, state_size])
        total_reward = 0
        episode_length = 0

        while True :#episode_length <= max_episode_length:
            # Exploration-exploitation trade-off
            if np.random.rand() <= epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(model.predict(state))
            #print(env.step(action))
            next_state, reward, done, _, _ = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])

            # Store experience in replay memory
            memory.append((state, action, reward, next_state, done))

            state = next_state
            total_reward += reward

            episode_length += 1
            if done:
                break

        # Experience replay
        if len(memory) > batch_size:
            # Sample a random batch from replay memory
            minibatch = np.array(random.sample(memory, batch_size))
            for state, action, reward, next_state, done in minibatch:
                target = reward if done else reward + 0.95 * np.max(model.predict(next_state))
                target_f = model.predict(state)
                target_f[0][action] = target
                model.fit(state, target_f, epochs=1, verbose=0)

        epsilon *= epsilon_decay
        rewards.append(total_reward)

    # Plot reward convergence graph
    plt.plot(rewards)
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.title('DQN Training Convergence')
    plt.show()

    return model, memory

In [10]:
# Initialize CartPole environment
env = gym.make('CartPole-v1')

# Train DQN policy
dqn_model, replay_memory = train_dqn(env,600)

dqn_model1, replay_memory1 = dqn_model, replay_memory

1/1 [==============================] - 0s 27ms/step


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (8, 5) + inhomogeneous part.

In [ ]:
print(len(replay_memory))

replay_memory = list(replay_memory)


print(replay_memory[0])